In [ ]:
!pip install hazm

In [1]:
import numpy as np
import pandas as pd
import re
from hazm import *

In [2]:
df = pd.read_csv('Snappfood - Sentiment Analysis.csv', encoding ='utf-8', sep='\t')
df

,Unnamed: 0,comment,label,label_id
0,NaN,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,NaN,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0
2,NaN,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0
3,NaN,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0.0
4,NaN,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0
...,...,...,...,...
69995,NaN,سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...,SAD,1.0
69996,NaN,سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...,SAD,1.0
69997,NaN,من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...,HAPPY,0.0
69998,NaN,همرو بعد ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو...,SAD,1.0


In [3]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('label_id', axis=1, inplace=True)

In [4]:
print(df['label'].value_counts())

HAPPY    34916
SAD      34564
1          436
0           84
Name: label, dtype: int64


In [5]:
df.drop(df[df['label'] == '0'].index, inplace=True)
df.drop(df[df['label'] == '1'].index, inplace=True)
print(df['label'].value_counts())

HAPPY    34916
SAD      34564
Name: label, dtype: int64


In [6]:
df['label'] = df['label'].astype('category')

In [7]:
# df_bkp = df.copy()
df

,comment,label
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY
4,شیرینی وانیلی فقط یک مدل بود.,HAPPY
...,...,...
69995,سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...,SAD
69996,سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...,SAD
69997,من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...,HAPPY
69998,همرو بعد ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو...,SAD


In [8]:
# a: preprocessing

lemmatizer = Lemmatizer()
normalizer = Normalizer()

def clean_txt(string):
    result = re.sub('','',string)          #remove HTML tags
    result = re.sub('https://.*','',result)   #remove URLs
    result = re.sub('[a-zA-Z#!?؟:$@%\.,\-()"*+-]', '',result)    #remove non-alphanumeric characters 
    
    words = normalizer.normalize(result)
    words = words.split(' ')
    
    for i in range(len(words)):
        words[i] = lemmatizer.lemmatize(words[i])
        
        
    return " ".join(words)

In [9]:
df['comment'] = df['comment'].apply(clean_txt)
df

,comment,label
0,واقعا حیف وقت که نوشت#نویس سرویس دهیتون شده اف...,SAD
1,قرار بود#باش ۱ ساعته برسه ولی نیم ساعت زود از ...,HAPPY
2,قیمت این مدل اصلا با کیفیت سازگار نداره، فقط ظ...,SAD
3,عالللی بود#باش همه چه درست و به اندازه و کیفیت...,HAPPY
4,شیرینی وانیل فقط یک مدل بود#باش,HAPPY
...,...,...
69995,سلام من به فاکتور غذا که سفارش میدم احتیاج داش...,SAD
69996,سایز پیتزا نسبت به سفارشاتی که قبلا گذشت#گذر ک...,SAD
69997,من قارچ اضافه رو اضافه کرده بود#باش بود#باش ام...,HAPPY
69998,همرو بعد ۲ساعت تاخیر اشتباه آوردن پول رفت#رو ر...,SAD


In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

comments = df['comment'].values
labels = df['label'].values

encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [12]:
train_comments, test_comments, train_labels, test_labels = train_test_split(comments, encoded_labels, stratify=encoded_labels)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict

vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(train_comments)
vocab = vec.get_feature_names_out()
X = X.toarray()

word_counts = {}
for l in range(2):
    word_counts[l] = defaultdict(lambda: 0)
    
    
for i in range(X.shape[0]):
    l = train_labels[i]
    for j in range(len(vocab)):
        word_counts[l][vocab[j]] += X[i][j]

In [16]:
def laplace_smoothing(n_label_items, vocab, word_counts, word, text_label):
    a = word_counts[text_label][word] + 1
    b = n_label_items[text_label] + len(vocab)
    
    return math.log(a/b)

In [19]:
def group_by_label(x, y, labels):
    data = {}
    for l in labels:
        data[l] = x[np.where(y == l)]
    return data
 

def fit(x, y, labels):
    n_label_items = {}
    log_label_priors = {}
    n = len(x)
    grouped_data = group_by_label(x, y, labels)
    for l, data in grouped_data.items():
        n_label_items[l] = len(data)
        log_label_priors[l] = math.log(n_label_items[l] / n)

    return n_label_items, log_label_priors

In [25]:
def predict(n_label_items, vocab, word_counts, log_label_priors, labels, x):
    result = []
    for text in x:
        label_scores = {l: log_label_priors[l] for l in labels}
        words = text.split(' ')
        
        for word in words:
            if word not in vocab: continue
                
            for l in labels:
                log_w_given_l = laplace_smoothing(n_label_items, vocab, word_counts, word, l)
                label_scores[l] += log_w_given_l
                
        result.append(max(label_scores, key=label_scores.get))
        
    return result

In [26]:
from sklearn.metrics import accuracy_score
import math

labels = [0,1]
n_label_items, log_label_priors = fit(train_comments, train_labels,labels)
pred = predict(n_label_items, vocab, word_counts, log_label_priors, labels, test_comments)

print("Accuracy of prediction on test set : ", accuracy_score(test_labels, pred))

Accuracy of prediction on test set :  0.7780080598733449
